# RDP Library Sessions Helper
The Refinitiv Data Platform Python Library provides the ability to access content from the desktop, directly to the cloud or through deployed platform services. Defined as a session, the user can choose how they want access the platform to execute all the examples within this package.  
The example code below demonstrates the difference between using a default session or multiple sessions as per your requirement.

**Note**: The *Deployed Platform* session only supports streaming services.

In [ ]:
# Configuration file and Platform Python Library packages
import configparser as cp
import refinitiv.dataplatform as rdp

### Configuration
Configure your environment based on your session of choice and credentials for that session.

Load up the *session.cfg* file that contains the various credential values for your environment. 

In [ ]:
def read_config(file):
    config = cp.ConfigParser()
    config.read(file)
    return config

### Using Multiple Sessions?
If you are going to be multiple session / connection types, get the requested session type and return it back to caller

In [ ]:
def get_session(target_session):
    config = read_config('session.cfg')
    print("Session choice: ",target_session)
    session_dict = {
        'desktop': get_desktop_session(config),
        'platform': get_platform_session('platform',config),
        'deployed': get_deployed_session(config)
    }
    session = session_dict.get(target_session)
    if session is None:
        raise Exception('Session not specified in configuration file')
    print(session)    
    return session

### Default Session
If you are only using a single session / connection type just open the session and set it to the default

In [ ]:
def open_session(target_session):
    config = read_config('session.cfg')
    print("Session choice: ",target_session)
    session_dict = {
        'desktop': get_desktop_session(config),
        'platform': get_platform_session('platform',config),
        'deployed': get_deployed_session(config)
    }
    session = session_dict.get(target_session)
    if session is None:
        raise Exception('Session not specified in configuration file')
    print(session)    
    session.open()
    rdp.set_default_session(session)
    return

In [ ]:
def get_desktop_session(config):
    app_key = config['desktop']['app_key']
    return rdp.DesktopSession(
                        app_key,
                        on_state = lambda session, state, message : print("Desktop session state: ", state, message),
                        on_event = lambda session, event, message : print("Desktop session event: ", event.message))

In [ ]:
def get_platform_session(target_session,config):
    app_key = config[target_session]['app_key']
    user = config[target_session]['user']
    password = config[target_session]['password']
    return rdp.PlatformSession(
                        app_key,
                        rdp.GrantPassword(
                            username = user,
                            password = password),
                        on_state = lambda session, state, message : print("Platform session state: ", state, message),
                        on_event = lambda session, event, message : print("Platform session event: ", event, message))

In [ ]:
def get_deployed_session(config):
    app_key = config['deployed']['app_key']
    host = config['deployed']['host']
    user = config['deployed']['user']
    return rdp.DeployedPlatformSession(
                        app_key = app_key,
                        host = host,
                        dacs_username = user,
                        on_state = lambda session, state, message : print("Deployed session state: ", state, message),
                        on_event = lambda session, event, message : print("Deployed session event: ", event, message))